# Walk 2019 Models

Models from L. Walk et al., [Phys. Rev. D 101:123013, 2019](https://arxiv.org/abs/1910.12971) (s15.0c) taken from the Garching Supernova archive (https://wwwmpa.mpa-garching.mpg.de/ccsnarchive/data/Walk2019/) with permission for use in SNEWPY.

Reference: L. Walk et al., *Neutrino emission characteristics of black hole formation in three-dimensional simulations of stellar collapse*, Phys. Rev. D 101:123013, 2019
- [doi:10.1103/PhysRevD.101.123013](https://journals.aps.org/prd/abstract/10.1103/PhysRevD.101.123013)
- [arXiv:1910.12971](https://arxiv.org/abs/1910.12971)

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

from astropy import units as u 

from snewpy.neutrino import Flavor, MassHierarchy
from snewpy.models.ccsn import Walk_2019
from snewpy.flavor_transformation import NoTransformation, AdiabaticMSW, ThreeFlavorDecoherence

mpl.rc('font', size=16)
%matplotlib inline

## Initialize Models

To start, let’s see what progenitors are available for the `Walk_2019` model. We can use the `param` property to view all physics parameters and their possible values:

In [ ]:
Walk_2019.param

We’ll initialise this progenitor. If this is the first time you’re using a progenitor, snewpy will automatically download the required data files for you.

In [ ]:
model = Walk_2019(progenitor_mass=40*u.solMass)

model

Finally, let’s plot the luminosity of different neutrino flavors for this model. (Note that the `Walk_2019` simulations didn’t distinguish between $\nu_x$ and $\bar{\nu}_x$, so both have the same luminosity.)

In [ ]:
fig, axes = plt.subplots(1, 1, figsize=(12, 5), sharex=True, sharey=True, tight_layout=True)

for flavor in Flavor:
    axes.plot(model.time, model.luminosity[flavor]/1e51,  # Report luminosity in units foe/s
            label=flavor.to_tex(),
            color='C0' if flavor.is_electron else 'C1',
            ls='-' if flavor.is_neutrino else ':',
            lw=2)
axes.set(xlim=(0, 0.6),
        xlabel=r'$t-t_{\rm bounce}$ [s]',
        title=r'{}: {} $M_\odot$'.format(model.metadata['EOS'], model.metadata['Progenitor mass'].value))
axes.grid()
axes.legend(loc='upper right', ncol=2, fontsize=18)
axes.set(ylabel=r'luminosity [foe s$^{-1}$]');

## Initial and Oscillated Spectra

Plot the neutrino spectra at the source and after the requested flavor transformation has been applied.

### Adiabatic MSW Flavor Transformation: Normal mass ordering

In [ ]:
# Adiabatic MSW effect. NMO is used by default.
xform_nmo = AdiabaticMSW()

# Energy array and time to compute spectra.
# Note that any convenient units can be used and the calculation will remain internally consistent.
E = np.linspace(0,100,201) * u.MeV
t = 50*u.ms

ispec = model.get_initial_spectra(t, E)
ospec_nmo = model.get_transformed_spectra(t, E, xform_nmo)

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(12,5), sharex=True, sharey=True, tight_layout=True)

for i, spec in enumerate([ispec, ospec_nmo]):
    ax = axes[i]
    for flavor in Flavor:
        ax.plot(E, spec[flavor],
                label=flavor.to_tex(),
                color='C0' if flavor.is_electron else 'C1',
                ls='-' if flavor.is_neutrino else ':', lw=2,
                alpha=0.7)

    ax.set(xlabel=r'$E$ [{}]'.format(E.unit),
           title='Initial Spectra: $t = ${:.1f}'.format(t) if i==0 else 'Oscillated Spectra: $t = ${:.1f}'.format(t))
    ax.grid()
    ax.legend(loc='upper right', ncol=2, fontsize=16)

ax = axes[0]
ax.set(ylabel=r'flux [s$^{-1}$]')

fig.tight_layout();